# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import api_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#pull csv from weatherpy
path = "../WeatherPY/city_summary_df.csv"
city_data = pd.read_csv(path)

city_data.head()

,Unnamed: 0,City Name,City Country,City ID,City Lat,City Long,Temps (F),Humidity %,Cloudiness %,Windspeed (mph),City Datetime
0,0,albany,US,5106841,42.60,-73.97,41.00,76,90,6.93,1580068688
1,1,piritiba,BR,3453494,-11.73,-40.56,74.16,85,100,4.70,1580068968
2,2,port alfred,ZA,964432,-33.59,26.89,63.00,78,28,7.00,1580068969
3,3,qaanaaq,GL,3831208,77.48,-69.36,-10.25,56,4,5.14,1580068677
4,4,punta arenas,CL,3874787,-53.15,-70.92,57.20,47,20,32.21,1580068971


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=api_key)


In [4]:
locations = city_data[["City Lat", "City Long"]]
weights = city_data['Humidity %']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#Narrow down the DataFrame to find your ideal weather condition. For example:

#  * A max temperature lower than 90 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.
#  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

city_name = []


for index, row in city_data.iterrows():
    if row["Cloudiness %"] == 0:
        if row ["Windspeed (mph)"] < 10:
            if row["Temps (F)"] >70 and row["Temps (F)"]<90:
                city_name.append(row)

city_weather_df = pd.DataFrame(city_name)

city_weather_df.head()
    

,Unnamed: 0,City Name,City Country,City ID,City Lat,City Long,Temps (F),Humidity %,Cloudiness %,Windspeed (mph),City Datetime
29,29,kavieng,PG,2094342,-2.57,150.80,83.16,74,0,5.46,1580068986
67,67,taoudenni,ML,2450173,22.68,-3.98,75.69,22,0,9.98,1580069013
128,128,castro,BR,3466704,-24.79,-50.01,79.25,48,0,0.81,1580069055
231,231,chiredzi,ZW,893485,-21.05,31.67,76.87,53,0,5.88,1580069119
235,235,iguape,BR,3461465,-24.71,-47.56,80.49,58,0,5.68,1580069121


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [0]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [0]:
# Add marker layer ontop of heat map


# Display Map